In [1]:
import pandas as pd
import string
import numpy as np
import glob
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer

In [2]:
porter=PorterStemmer()
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
import pickle
with open('tokenizer-200-nou.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
def clean_punct(a):        
    return a.lower().translate(str.maketrans('', '', string.punctuation))

def clean_stop(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
        if tk in stopwords:
            continue
        else:
            str1 += tk+" "
            
    return str1

def clean_stem(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
            str1 += porter.stem(tk)+" "
            
    return str1

In [6]:
from tensorflow import keras
model = keras.models.load_model('./Model-200-drop75')

count=0;

pozitive_total=0
negative_total=0
pozitive_corecte=0
negative_corecte=0
neutre_total=0
neutre_corecte=0
txtfiles = []
for file in glob.glob("Stiri_pozitive/*.txt"):
    txtfiles.append(file)
    

print("---Stiri Pozitive---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
        
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        pozitive_total+=1
        if sum_neg < sum_poz:
            pozitive_corecte+=1

     
print("Total stiri pozitive : "+str(pozitive_total))
print("Corecte stiri pozitive : "+str(pozitive_corecte)) 
print("Acuratete Pozitive: "+str(pozitive_corecte/pozitive_total))
print("")    

txtfiles = []
for file in glob.glob("Stiri_negative/*.txt"):
    txtfiles.append(file)
print("---Stiri Negative---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        #paragrafe=list(map(clean_stem, paragrafe))
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
                
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        negative_total+=1
        if sum_neg > sum_poz:
            negative_corecte+=1
    
print("Total stiri negative : "+str(negative_total))
print("Corecte stiri negative : "+str(negative_corecte)) 
print("Acuratete negative: "+str(negative_corecte/negative_total))

print("")
print("Total stiri :" + str(pozitive_total+negative_total+neutre_total))
print("Total corecte :" + str(pozitive_corecte+negative_corecte+neutre_corecte))
print("Acuratete :" + str((pozitive_corecte+negative_corecte+neutre_corecte)/(pozitive_total+negative_total+neutre_total)))
    
    

---Stiri Pozitive---
Stiri_pozitive/34.txt  0.17780022825300693     0.8221997729937236
Stiri_pozitive/12.txt  0.4206861289120034     0.579313881719779
Stiri_pozitive/41.txt  0.26040322401095184     0.7395967743669947
Stiri_pozitive/11.txt  0.5067932595224941     0.4932067603749387
Stiri_pozitive/31.txt  0.26268214451676086     0.7373178639537818
Stiri_pozitive/3.txt  0.186963354105415     0.8130366452499399
Stiri_pozitive/39.txt  0.39284833493325994     0.6071516620974378
Stiri_pozitive/17.txt  0.1365983877139978     0.8634016131743406
Stiri_pozitive/8.txt  0.25234914883257037     0.7476508661289096
Stiri_pozitive/10.txt  0.24870510007429697     0.7512948964604267
Stiri_pozitive/16.txt  0.40061412723734974     0.5993858799726391
Stiri_pozitive/23.txt  0.14187804894297534     0.8581219820295953
Stiri_pozitive/33.txt  0.02274383392150157     0.9772561482179944
Stiri_pozitive/30.txt  0.18317446720358238     0.8168255494171212
Stiri_pozitive/18.txt  0.34925425920454     0.6507457470825545
